# XAJNet
使用神经网络建模新安江模型

In [ ]:
# 自动加载模块
%load_ext autoreload
%autoreload 2

## 1. 数据处理

In [ ]:
from hydronetwork.data.camels_us import load_timeseries
from hydronetwork.dataset.preprocessing import split_timeseries
from hydronetwork.dataset.dataset import get_dataset

lookback = 365
horizon = 7
timeseries = load_timeseries(gauge_id='13023000', unit="mm/day")
timeseries.columns

对['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)']进行归一化

In [ ]:
from hydronetwork.dataset.preprocessing import normalize

timeseries_need_normalize = timeseries[['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)']]
timeseries_normalized = normalize(timeseries_need_normalize)
timeseries = timeseries.drop(columns=['srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'])
timeseries = timeseries.join(timeseries_normalized)

构建数据集

In [ ]:
train_data, test_data = split_timeseries(timeseries, split_list=[0.8, 0.2])
train_dataset = get_dataset(train_data,
                            lookback=lookback,
                            horizon=horizon,
                            batch_size=512,
                            features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                            features_lookback=["streamflow"],
                            target="streamflow"
                            )
test_dataset = get_dataset(test_data,
                           lookback=lookback,
                           horizon=horizon,
                           batch_size=512,
                           features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
                           features_lookback=["streamflow"],
                           target="streamflow"
                           )
# dataset = get_dataset(timeseries,
#                       lookback=lookback,
#                       horizon=horizon,
#                       batch_size=512,
#                       features_bidirectional=['prcp(mm/day)', 'srad(W/m2)', 'tmax(C)', 'tmin(C)', 'vp(Pa)'],
#                       target="streamflow"
#                       )

## 2. 模型构建

In [ ]:
from hydronetwork.model.xinanjiang_net.xinanjiang_net import XinAnJiangNet

n=64
model = XinAnJiangNet(n=n)

## 3. 模型训练

In [8]:
from hydronetwork.train import WarmupExponentialDecay, callback_for_features_selection
from hydronetwork.evaluate.metrics import rmse, nse
from keras.src.optimizers import AdamW

model.compile(optimizer=AdamW(learning_rate=WarmupExponentialDecay(dataset_length=len(train_dataset),
                                                                   initial_learning_rate=1e-4, )),
              loss=nse,
              metrics=[rmse]
              )
model.fit(train_dataset,
          epochs=500,
          verbose=1,
          shuffle=True,
          validation_data=test_dataset,
          callbacks=callback_for_features_selection())

C:\Users\18313\miniconda3\envs\Hydro\Lib\site-packages\keras\src\layers\layer.py:1381: UserWarning: Layer 'xin_an_jiang_net_1' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Exception encountered when calling StoredFullRunoffCell.call().

Cannot copy out of meta tensor; no data!

Arguments received by StoredFullRunoffCell.call():
  • precipitation=torch.Tensor(shape=torch.Size([128]), dtype=float32)
  • evaporation_features=torch.Tensor(shape=torch.Size([128

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 3767.1245 - rmse: 2.4124 - val_loss: 3618.5305 - val_rmse: 2.7135
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 2100.7153 - rmse: 2.2651 - val_loss: 3603.6953 - val_rmse: 2.7090
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 2489.9082 - rmse: 2.2951 - val_loss: 3576.4827 - val_rmse: 2.7009
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 5235.7803 - rmse: 2.1202 - val_loss: 3540.7739 - val_rmse: 2.6902
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - loss: 2039.7738 - rmse: 2.2449 - val_loss: 3499.0869 - val_rmse: 2.6777
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - loss: 1910.4835 - rmse: 2.0926 - val_loss: 3454.8298 - val_rmse: 2.6644
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 2369.0276 - rmse: 2.2975 - val_loss: 3404.9685 - val_rmse: 2.6494
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - loss: 2270.8169 - rmse: 2.2768 - val_loss: 3361.3088 - val_rmse: 2.6362


KeyboardInterrupt: 